In [2]:
# !wget -O en-fa.tmx.gz https://opus.nlpl.eu/download.php?f=TEP/v1/tmx/en-fa.tmx.gz
!wget -O en-fa.txt.zip https: // opus.nlpl.eu / download.php?f=TEP / v1 / moses / en-fa.txt.zip
# !gzip -dk en-fa.tmx.gz
!unzip en-fa.txt.zip

--2022-07-14 08:08:29--  https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip [following]
--2022-07-14 08:08:31--  https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16353318 (16M) [application/zip]
Saving to: ‘en-fa.txt.zip’

en-fa.txt.zip       100%[===================>]  15.60M  5.43MB/s    in 2.9s    

2022-07-14 08:08:36 (5.43 MB/s) - ‘en-fa.txt.zip’ saved [16353318/16353318]

Archive:  en-fa.txt.zip
  inflating: TEP.en-fa.en            
  inflating: TEP.en-fa.fa            
  inflating: README    

In [ ]:
!cat TEP.en-fa.en | grep -a "?"

In [ ]:
!pip install "tensorflow-text==2.8.*"

In [5]:
import pathlib
import re
import string

import numpy as np
import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow import keras

In [6]:
text_file = keras.utils.get_file(
    fname="fa-eng.zip",
    origin="https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip",
    extract=True,
)

text_fa = pathlib.Path(text_file).parent / "TEP.en-fa.fa"
text_en = pathlib.Path(text_file).parent / "TEP.en-fa.en"

16367616/16353318 [==============================] - 3s 0us/step


In [7]:
with open(text_en) as f:
    lines_en = f.read().split("\n")[:-1]

with open(text_fa) as f:
    lines_fa = f.read().split("\n")[:-1]

# text_persian = []

# for line in range(len(lines_en)):
#   eng = lines_en[line]
#   fa = lines_fa[line]
#   text_persian.append((eng, fa))


In [8]:
index = 10
print(lines_en[index])
print(lines_fa[index])

assert len(lines_en) == len(lines_fa)
len(lines_en)

its the last feed weve got .
اين آخرين علوفه اي بود که ما داشتيم .


612086

In [9]:
inp = lines_en
targ = lines_fa

BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [10]:
for example_input_batch, example_target_batch in dataset.take(1):
    print(example_input_batch[1].numpy())
    print()
    print(example_target_batch[1].numpy().decode())
    print(example_target_batch[1].numpy())

    break

b'to the chapter of ethics.'

به فصل اخلاقيات
b'\xd8\xa8\xd9\x87 \xd9\x81\xd8\xb5\xd9\x84 \xd8\xa7\xd8\xae\xd9\x84\xd8\xa7\xd9\x82\xd9\x8a\xd8\xa7\xd8\xaa'


In [12]:
def standardize_en(text):
    # Split accecented characters.
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')

    return text


def standardize_fa(text):
    strip_chars = re.sub('[\[\].?!,¿]', '', string.punctuation)

    text = tf.strings.regex_replace(text, f"[{re.escape(strip_chars)}]", '')

    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')

    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')

    return text

In [13]:
example_text = tf.constant("what is it? the captain has escaped.")
print(example_text.numpy().decode())
print(standardize_en(example_text).numpy().decode())

what is it? the captain has escaped.
[START] what is it ?  the captain has escaped . [END]


In [14]:
strip_chars = re.sub('[\[\].?!,¿]', '', string.punctuation)
strip_chars

'"#$%&\'()*+-/:;<=>@\\^_`{|}~'

In [15]:
re.escape(strip_chars)

'"\\#\\$%\\&\'\\(\\)\\*\\+\\-/:;<=>@\\\\\\^_`\\{\\|\\}\\~'

In [16]:
max_vocab_size = 50000

en_vectorization = tf.keras.layers.TextVectorization(
    standardize=standardize_en,
    max_tokens=max_vocab_size
)

fa_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_size,
    standardize=standardize_fa
)

In [17]:
en_vectorization.adapt(inp)
en_vectorization.get_vocabulary()[:10]

['', '[UNK]', '.', '[START]', '[END]', ',', 'you', 'the', 'i', 'to']

In [18]:
fa_vectorization.adapt(targ)
fa_vectorization.get_vocabulary()[:10]

['', '[UNK]', '[START]', '[END]', '.', '،', 'من', 'به', 'را', 'تو']

In [29]:
en_example_tokens = en_vectorization(example_input_batch)
en_example_tokens[:3]

<tf.Tensor: shape=(3, 21), dtype=int64, numpy=
array([[    3,     1,     2,     4,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    3,     9,     7,  4707,    13, 10519,     2,     4,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    3,     6,    88,  3861,   119,    17,     7,   910,    13,
           10,  4158,    11,   283,     9,   188,    60,    12,     2,
            4,     0,     0]])>

In [30]:
fa_example_tokens = fa_vectorization(example_target_batch)
fa_example_tokens[:3]

<tf.Tensor: shape=(3, 19), dtype=int64, numpy=
array([[    2,     1,     4,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    2,     7,  2967, 19685,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    2,     9,   416,  1166,  6117,  2488,   750,    10,    16,
           13,   103,   113,   750,     4,     3,     0,     0,     0,
            0]])>

In [20]:
len(fa_vectorization.get_vocabulary())

50000

In [32]:
input_vocab = np.array(en_vectorization.get_vocabulary())
tokens = input_vocab[en_example_tokens[2].numpy()]
' '.join(tokens)

'[START] you cant kneel down in the middle of a highway and live to talk about it . [END]  '

In [35]:
input_vocab = np.array(fa_vectorization.get_vocabulary())
tokens = input_vocab[fa_example_tokens[2].numpy()]
print(tokens)
' '.join(tokens)

['[START]' 'تو' 'نميتوني' 'وسط' 'بزرگراه' 'زانو' 'بزني' 'و' 'در' 'اين'
 'مورد' 'حرف' 'بزني' '.' '[END]' '' '' '' '']


'[START] تو نميتوني وسط بزرگراه زانو بزني و در اين مورد حرف بزني . [END]    '

In [ ]:
!head -n20 TEP.en-fa.en

raspy breathing .
dad .
maybe its the wind .
no .
stop please stop .
you have a week , evans then well burn the house .
william .
god damn it , william .
god damn it put that down .
let go .
its the last feed weve got .
ill take care of this .
no , you wont .
herds over the ridge by now . you go get cleaned up .
you lied to me , dan .
you told me we made payments to hollander . we did .
so , how do you think we bought feed , alice . three months water , the medicine for mark .
i had a choice between our family and paying them down .
were supposed to make decisions together .
would you have made anything different .


In [ ]:
!head -n20 TEP.en-fa.fa

صداي خر خر .
پدر .
شايد صداي باد باشه .
نه .
دست نگه داريد خواهش ميکنم دست نگه داريد .
اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواهيم سوزوند .
ويليام .
لعنتي . ويليام 8 .
لعنت به تو . اونو بذار زمين .
بذار برم .
اين آخرين علوفه اي بود که ما داشتيم .
من خودم اين کارو انجام ميدهم .
نه تو نميکني .
گله را آوردم بيرون الان . تو برو اونجا را تميز کن .
تو به من دروغ گفتي دن .
تو به من گفتي قرضمونو به هلندر پرداخت کرديم . ما پرداخت کرديم .
خب پس تو فکر ميکني ما چطوري علوفه خريديم سه ماه آب خرج دارو مارک .
من فقط يک انتخاب بين خانوادم و پرداخت قرض اونا داشتم .
ما قرار بود تصميماتو با هم بگيريم .
مگه تو تصميمه ديگه اي ميگرفتي .


In [ ]:
!cat TEP.en-fa.fa | wc -l
!cat TEP.en-fa.en | wc -l

612086
612086


In [ ]:
!head -n30 en-fa.tmx

<?xml version="1.0" encoding="UTF-8" ?>
<tmx version="1.4">
<header creationdate="Sun Mar  4 00:48:54 2018"
          srclang="en"
          adminlang="en"
          o-tmf="unknown"
          segtype="sentence"
          creationtool="Uplug"
          creationtoolversion="unknown"
          datatype="PlainText" />
  <body>
    <tu>
      <tuv xml:lang="en"><seg>raspy breathing .</seg></tuv>
      <tuv xml:lang="fa"><seg>صداي خر خر .</seg></tuv>
    </tu>
    <tu>
      <tuv xml:lang="en"><seg>maybe its the wind .</seg></tuv>
      <tuv xml:lang="fa"><seg>شايد صداي باد باشه .</seg></tuv>
    </tu>
    <tu>
      <tuv xml:lang="en"><seg>stop please stop .</seg></tuv>
      <tuv xml:lang="fa"><seg>دست نگه داريد خواهش ميکنم دست نگه داريد .</seg></tuv>
    </tu>
    <tu>
      <tuv xml:lang="en"><seg>you have a week , evans then well burn the house .</seg></tuv>
      <tuv xml:lang="fa"><seg>اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواهيم سوزوند .</seg></tuv>
    </tu>
    <tu>
      <tuv 

In [ ]:
!cat en-fa.tmx | grep "" -B 1